In [ ]:
from cdlib import algorithms, evaluation
from cdlib import TemporalClustering
import matplotlib.pyplot as plt

import pandas as pd
import networkx as nx
import pickle

In [ ]:
#Per plottare le communities per dimensione
def plot_communities_dimentions(coms):
    myd = {}
    for t in coms:
        lunghezza = len(t)

        if str(lunghezza) in myd:
            myd[str(lunghezza)] = myd[str(lunghezza)] + 1
        else:
            myd[str(lunghezza)] = 1

    print(myd)
    plt.figure(figsize=(20,5))
    plt.bar(myd.keys(), myd.values(), color='b')

In [ ]:
coms_months = []
for i in range(12):
    with open(f'CommunityDiscovery/communities_{i}.pickle', 'rb') as f:
        louvain_coms = pickle.load(f)
    coms_months.append(louvain_coms

In [ ]:
for i in coms_months:
    
    max_c=0
    for com in i.communities:
        if(len(com)>max_c):
            max_c = len(com)
    print(str(len(i.communities)) + " - " + str(max_c))

In [ ]:
tc = TemporalClustering()
for t in range(12):
    tc.add_clustering(coms_months[t], t)

In [ ]:
tc.get_observation_ids()

Una semplice misura della stabilità temporale è il "clustering stability trend"
Dato come input un TemporalClustering e un punteggio di somiglianza delle partizioni (ad esempio, NMI, NF1...) tale trend descrive quante partizioni tendono a rimanere le stesse con il passare del tempo.

L'indice di stabilità viene calcolato per coppie di cluster temporalmente adiacent

In [ ]:
trend = tc.clustering_stability_trend(evaluation.nf1)
trend

Poiché il nostro obiettivo è trasformare un algoritmo statico in uno dinamico, una volta calcolati i clustering dobbiamo abbinarli tra id temporali consecutivi.

In [ ]:
tc.has_explicit_match()

Dall'approccio personalizzato non è venuto fuori però un abbinamento esplicito. Si definisce una funzione di somiglianza (prendendo come input due set e restituendo un valore float) per eseguire tale corrispondenza intertemporale

In [ ]:
# una funzione di similarità il cui co-dominio è in [0, 1]
jaccard = lambda x, y: len(set(x) & set(y)) / len(set(x) | set(y))

Successivamente possiamo applicarlo alla nostra partizione temporale, specificando se deve essere applicato solo dal passato al futuro (un lato) o anche dal futuro al passato (due lati).

Il primo scenario consente di identificare eventi di merge, il secondo anche di split.

In [ ]:
matches = tc.community_matching(jaccard, two_sided=True)

community_matching restituisce una lista di tuple della forma:
(Ti_Ca, TjCb, score)
I nomi di comunità vengono assegnati seguendo lo schema {tid}{cid}, dove tid è il tempo di osservazione e cid è la posizione della comunità all'interno dell'oggetto Clustering.

Una volta ottenuta la corrispondenza tra le comunità, possiamo calcolare il loro grafo del ciclo di vita: un polytree diretto che rappresenta la trasformazione della comunità nel tempo

In [ ]:
com_polytree = tc.lifecycle_polytree(jaccard, two_sided=True)

In [ ]:
edges = list(com_polytree.edges(data=True))

In [ ]:
edges[0]

In [ ]:
new = com_polytree.edge_subgraph([edges[0][0:2]]).copy()

In [ ]:
new.edges(data=True)

In [ ]:
selelected_edges = []

In [ ]:
for i in edges:
    if i[2]["weight"] > 0:
        selelected_edges.append(i[0:2])
new_polytree = com_polytree.edge_subgraph(selelected_edges).copy()

In [ ]:
for nodes in nx.weakly_connected_components(new_polytree):
    sub =  com_polytree.subgraph(nodes)
    nx.draw(sub, pos=nx.spring_layout(sub), with_labels=True)

In [ ]:
n_nodes = []
month_com = []
for i in list(new_polytree.nodes()):
    month_com.append(int(i.split("_")[0]))
    n_nodes.append(i)

In [ ]:
s_edges = []
t_edges = []
for i in list(new_polytree.edges()):
    s_edges.append(i[0])
    t_edges.append(i[1])

In [ ]:
new_df = pd.DataFrame({"Id": n_nodes, "month": month_com})
new_df.to_csv("nodes.csv", index=False)

In [ ]:
new_df = pd.DataFrame({"Source": s_edges, "Target": t_edges})
new_df.to_csv("edges.csv", index=False)

In [ ]:
nx.write_graphml(new_polytree, "polytree_threshold.graphml")

In [ ]:
def birth(nodes, edges):
    n_birth = 0
    birth_nodes = []
    for node in nodes:
        is_birth = True
        for edge in edges:
            if node == edge[1]:
                is_birth = False
                break
        if is_birth:
            n_birth+=1
            birth_nodes.append(node)
    return n_birth, birth_nodes

In [ ]:
def death(nodes, edges):
    n_death = 0
    death_nodes = []
    for node in nodes:
        is_death = True
        for edge in edges:
            if node == edge[0]:
                is_death = False
                break
        if is_death:
            n_death+=1
            death_nodes.append(node)
    return n_death, death_nodes



In [ ]:
def merge(nodes, edges):
    n_merge = 0
    merge_nodes = {}
    for node in nodes:
        tmp_from = []
        n_from = 0
        for edge in edges:
            if node == edge[1]:
                tmp_from.append(edge[0])
                n_from+=1
        if n_from >1:
            n_merge+=1
            merge_nodes[node]=tmp_from
    return n_merge, merge_nodes

In [ ]:
def split(nodes, edges):
    n_split = 0
    split_nodes = {}
    for node in nodes:
        tmp_to = []
        n_to = 0
        for edge in edges:
            if node == edge[0]:
                tmp_to.append(edge[1])
                n_to+=1
        if n_to >1:
            n_split+=1
            split_nodes[node]=tmp_to
    return n_split, split_nodes

In [ ]:
def continue_(nodes, edges):
    n_continue = 0
    continue_nodes = []
    for node in nodes:
        n_to = 0
        tmp_to = ""
        for edge in edges:
            if node == edge[0]:
                tmp_to=edge[1]
                n_to+=1
        if n_to ==1:
            n_in = 0
            for edge in edges:
                if edge[1] == tmp_to:
                    n_in+=1
            if n_in ==1:
                n_continue+=1
                continue_nodes.append((node, tmp_to))
    return n_continue, continue_nodes

In [ ]:
def type_continue(continue_nodes, tc):
    grow = 0
    contraction = 0
    grow_ll = []
    contraction_ll = []
    for u,v in continue_nodes:
        u_  = u.split("_")
        v_  = v.split("_")
        com_u = tc.get_clustering_at(int(u_[0])).named_communities[u]
        com_v = tc.get_clustering_at(int(v_[0])).named_communities[v]
        l_u = len(com_u)
        l_v = len(com_v)
        if l_u < l_v:
            grow+=1
            grow_ll.append((u, v))
        if l_u > l_v:
            contraction+=1
            contraction_ll.append((u, v))
    return grow, contraction, grow_ll, contraction_ll

In [ ]:
def resourge(nodes, edges, tc):
    #nodes without exit edges
    no_exit_nodes = []
    for node in nodes:
        no_exit = True
        for edge in edges:
            if node == edge[0]:
                no_exit = False
                break
        if no_exit:
            no_exit_nodes.append(node)
    #nodes withou entry edges
    no_entry_nodes = []
    for node in nodes:
        no_entry = True
        for edge in edges:
            if node == edge[1]:
                no_entry = False
                break
        if no_entry:
            no_entry_nodes.append(node)
    #for all no_exit (at t):
    #       if there is a no_entry (at t + n) == no_exit (at t)
    #              resourge found
    resourge = 0
    resourge_ll = []
    for n_ex in no_exit_nodes:
        clust_ex, commu_ex = n_ex.split("_")
        for n_en in no_entry_nodes:
            clust_en, commu_en = n_en.split("_")
            if int(clust_en) > int(clust_ex):
                # from t to t+n
                com_ex = tc.get_clustering_at(int(clust_ex)).named_communities[n_ex]
                com_en = tc.get_clustering_at(int(clust_en)).named_communities[n_en]
                com_ex.sort()
                com_en.sort()
                if com_ex == com_en:
                    resourge+=1
                    resourge_ll.append((n_ex, n_en))
    return resourge, resourge_ll



In [ ]:
nodes_polytree = list(com_polytree.nodes())
edges_polytree = list(com_polytree.edges())

In [ ]:
print(len(nodes_polytree))
print(len(edges_polytree))

In [ ]:
nodes_new = list(new_polytree.nodes())
edges_new = list(new_polytree.edges())

In [ ]:
print(len(nodes_new))
print(len(edges_new))

In [ ]:
for i in new_polytree.edges(data=True):
    print(i)

In [ ]:
n_birth, birth_nodes = birth(nodes_polytree, edges_polytree)
print(n_birth)

In [ ]:
n_death, death_nodes = death(nodes_polytree, edges_polytree)
print(n_death)

In [ ]:
n_merge, merge_nodes = merge(nodes_polytree, edges_polytree)
print(n_merge)



In [ ]:
n_split, split_nodes = split(nodes_polytree, edges_polytree)
print(n_split)



In [ ]:
n_continue, continue_nodes = continue_(nodes_polytree, edges_polytree)
print(n_continue)



In [ ]:
if n_continue > 0:
    n_grow, n_contraction = type_continue(continue_nodes, tc)
    print("N grow events: ", n_grow)
    print("N contraction events: ", n_contraction)



In [ ]:
n_birth, birth_nodes = birth(nodes_new, edges_new)
print(n_birth)

In [ ]:
real_birth = 0
for i in birth_nodes:
    if i.split("_")[0] != "0":
        real_birth+=1
print(real_birth)



In [ ]:
n_death, death_nodes = death(nodes_new, edges_new)
print(n_death)

In [ ]:
real_death = 0
for i in death_nodes:
    if i.split("_")[0] != "11":
        real_death+=1
print(real_death)



In [ ]:
n_merge, merge_nodes = merge(nodes_new, edges_new)
print(n_merge)



In [ ]:
n_split, split_nodes = split(nodes_new, edges_new)
print(n_split)

In [ ]:
n_continue, continue_nodes = continue_(nodes_new, edges_new)
print(n_continue)



In [ ]:
if n_continue > 0:
    n_grow, n_contraction, grow_ll, contraction_ll = type_continue(continue_nodes, tc)
    print("N grow events: ", n_grow)
    print("N contraction events: ", n_contraction)



In [ ]:
com_8_27 = tc.get_clustering_at(8).named_communities["8_27"]
print(len(com_8_27))

In [ ]:
com_9_16 = tc.get_clustering_at(9).named_communities["9_16"]
print(len(com_9_16))

In [ ]:
com_10_37 = tc.get_clustering_at(10).named_communities["10_37"]
print(len(com_10_37))

In [ ]:
tc.get_clustering_at(1)



In [ ]:
max_grow = 0 
edge_grow = ("0", "0")
for i in grow_ll:
    tt = i[0].split("_")
    com_ = tc.get_clustering_at(int(tt[0])).named_communities[i[0]]
    if len(com_) > max_grow:
        max_grow = len(com_)
        edge_grow = i
print(edge_grow)



In [ ]:
com_10_3 = tc.get_clustering_at(10).named_communities["10_3"]
print(len(com_1_17))
com_11_2 = tc.get_clustering_at(11).named_communities["11_2"]
print(len(com_11_2))

In [ ]:
com_1_4 = tc.get_clustering_at(1).named_communities["1_4"]
print(len(com_1_4))

In [ ]:
com_1_17 = tc.get_clustering_at(1).named_communities["1_17"]
print(len(com_1_17))



In [ ]:
com_2_3 = tc.get_clustering_at(2).named_communities["2_3"]
print(len(com_2_3))



In [ ]:
com_3_4 = tc.get_clustering_at(3).named_communities["3_4"]
print(len(com_3_4))
com_3_36 = tc.get_clustering_at(3).named_communities["3_36"]
print(len(com_3_36))
com_3_32 = tc.get_clustering_at(3).named_communities["3_32"]
print(len(com_3_32))
com_3_44 = tc.get_clustering_at(3).named_communities["3_44"]
print(len(com_3_44))
com_3_15 = tc.get_clustering_at(3).named_communities["3_15"]
print(len(com_3_15))
com_3_43 = tc.get_clustering_at(3).named_communities["3_43"]
print(len(com_3_43))

In [ ]:
n_resourge, resourge_ll = resourge(nodes_new, edges_new, tc)



In [ ]:
n_resourge

